<a href="https://colab.research.google.com/github/GusSampaio/Classificacao_Multirrotulo_no_Dominio_Juridico/blob/main/Minera%C3%A7%C3%A3o_de_Textos_para_Dom%C3%ADnio_Jur%C3%ADdico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto Extensionista - Classificação Multirrótulo no Domínio Jurídico

O objetivo deste projeto é desenvolver uma solução de classificação multirrótulo aplicada ao domínio jurídico. Em particular, trabalharemos com processos que podem estar associados a múltiplos assuntos, o que exige uma abordagem específica para lidar com esse tipo de dados.


In [12]:
# Baixando e importando bibliotecas necessarias
%pip install iterative-stratification
%pip install -U sentence-transformers
%pip install scikit-multilearn-ng

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from torch.utils.data import DataLoader
import gc
from scipy import sparse

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils.class_weight import compute_class_weight
from transformers import AutoTokenizer, AutoModel, AutoModelForPreTraining
import concurrent.futures

from skmultilearn.adapt import MLkNN
from sklearn.metrics import f1_score, classification_report

from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Leitura da Base de Dados

In [2]:
!gdown 1E_aecYDw69UgYbgzQCjtP9rWonwpyFuJ
!gdown 1--unYhhi0HoMFT4kNyf91WwckjlTedYo

Downloading...
From (original): https://drive.google.com/uc?id=1E_aecYDw69UgYbgzQCjtP9rWonwpyFuJ
From (redirected): https://drive.google.com/uc?id=1E_aecYDw69UgYbgzQCjtP9rWonwpyFuJ&confirm=t&uuid=34c05d47-2aa9-4282-90b0-20d4aa4c03e9
To: /content/X_train.pkl
100% 878M/878M [00:13<00:00, 63.3MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1--unYhhi0HoMFT4kNyf91WwckjlTedYo
From (redirected): https://drive.google.com/uc?id=1--unYhhi0HoMFT4kNyf91WwckjlTedYo&confirm=t&uuid=1cbf5449-f6ef-43a9-8d4c-16786e98ee04
To: /content/Y_train.pkl
100% 138M/138M [00:02<00:00, 67.1MB/s]


In [3]:
X_train = pd.read_pickle("X_train.pkl")
y_train = pd.read_pickle("Y_train.pkl")

X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

## Análise Exploratória Descritiva

In [ ]:
X_train.head()

,texto
0,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. SERVID...
1,DECISÃO: - Vistos. Trata-se de agravo regiment...
2,DECISÃO: O Tribunal a quo assim dirimiu a cont...
3,DECISÃO EMBARGOS DECLARATÓRIOS – OMISSÃO. 1. P...
4,"DECISÃO: 1. Trata-se de habeas corpus,..."


In [ ]:
x_train_size = X_train.shape[0]
print("Tamanho do conjunto de treino:",x_train_size,'textos diferentes')

Tamanho do conjunto de treino: 220457 textos diferentes


In [ ]:
y_train.head()

,DIREITO DO TRABALHO,SISTEMA REMUNERATORIO E BENE,RMI RENDA MENSAL INICIAL REAJUSTES E REVISOES ESPECIFICAS,ATOS PROCESSUAIS,RESPONSABILIDADE CIVIL,COFINSDIREITO TRIBUTARIO,DIREITO PROCESSUAL PENAL,ATOS ADMINISTRATIVOS,CONTRIBUICOES PREVIDENCIARIAS,LIMITACOES AO PODER DE TRIBUTAR,...,CRIMES DE LAVAGEM OU OCULTACAO DE BE,DIREITO ADMINISTRATIVO E OUTRAS MATERIAS DE DIREITO PUBLICO,APOSENTADORIA,RESCISAO DO CONTRATO DE TRABALHO,CRIMES PREVISTOS NA LEGISLACAO EXTRAVAGANTE,BENEFICIOS EM ESPECIE,PARTES E PROCURADORES,MAGISTRATURA,PRISAO PREVENTIVA,EXCESSO DE PRAZO PARA INSTRUCAO / JULGAMENTO
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
print("Exemplo de texto jurídico:")
display(X_train.loc[0].texto)

print("\nClasses atribuídas a ele:")
for index, row in pd.DataFrame(y_train.loc[0]).iterrows():
  if row[0] != 0:
    print(row.name)

Exemplo de texto jurídico:


'EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. SERVIDOR PÚBLICO. LICENÇA-GESTANTE. EXONERAÇÃO. C.F., art. 7º, XVIII; ADCT, art. 10, II, b. I. - Servidora exonerada quando no gozo de licença-gestante: a exoneração constitui ato arbitrário, porque contrária à norma constitucional: C.F., art. 7º, XVIII; ADCT, art. 10, II, b. II. Negativa de trânsito ao RE. DECISÃO: - Vistos. Assim a ementa do acórdão recorrido da 2ª Câmara Especializada Cível do Tribunal de Justiça do Estado do Piauí: "A servidora pública, ainda que não concursada, não pode ser demitida no período dos cinco meses após o parto, porquanto amparado pela ESTABILIADE PROVISÓRIA, assegurada constitucionalmente a toda e qualquer trabalhadora-mãe, independentemente do regime jurídico de trabalho, principalmente quando comprovada a sua condição de contribuinte da Previdência Social do Estado - IAPEP. Decisão unânime, de acordo com o parecer ministerial" (fl. 130). Daí o RE, interposto pelo ESTADO DO PIAUÍ, fundado no art. 102, III, a, d


Classes atribuídas a ele:
REGIME ESTATU
SERVIDOR PUBLICO CIVIL
DIREITO ADMINISTRATIVO E OUTRAS MATERIAS DE DIREITO PUBLICO


In [ ]:
# Distribuição das classes.
y_train.sum().plot.bar(figsize=(15, 3))
plt.show()

# Separando um conjunto de validação

In [4]:
mskf = MultilabelStratifiedKFold(n_splits=2, shuffle=True, random_state=0)

for train_index, test_index in mskf.split(X_train, y_train):
   print("TRAIN:", train_index, "TEST:", test_index)
   X_train, X_val = X_train.iloc[train_index], X_train.iloc[test_index]
   y_train, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
   break

TRAIN: [     0      1      4 ... 220453 220454 220456] TEST: [     2      3      5 ... 220449 220450 220455]


In [ ]:
len(y_train), len(y_val)

(110186, 110271)

# Pré-processamento



In [ ]:
# Criando representações vetoriais (em formato de matriz TF-IDF) a partir dos textos
nltk.download("stopwords")

vectorizer = TfidfVectorizer(
    stop_words=stopwords.words("portuguese"),
    min_df=5
)

X_train_tf = vectorizer.fit_transform(X_train.texto.to_list())
X_val_tf = vectorizer.transform(X_val.texto.to_list())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
# Instanciando modelo gerador de embeddings multilinguais
multilingual_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Sugiro uso de GPUs se possível
X_train_multilingual_embeddings = multilingual_model.encode(
    X_train.texto.to_list(),
    show_progress_bar=True,
    normalize_embeddings=True
)

X_val_multilingual_embeddings = multilingual_model.encode(
    X_val.texto.to_list(),
    show_progress_bar=True,
    normalize_embeddings=True
)

# np.savez_compressed("X_train_multilingual_embeddings.npz", X_train_multilingual_embeddings)
# np.savez_compressed("X_val_multilingual_embeddings.npz", X_val_multilingual_embeddings)

# Se já possuir as embeddings em formato de arquivo .py comente as linhas acima e
# descomente a seguir
# X_train_multilingual_embeddings = np.load("X_train_multilingual_embeddings.npz")["arr_0"]
# X_val_multilingual_embeddings = np.load("X_val_multilingual_embeddings.npz")["arr_0"]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3444 [00:00<?, ?it/s]

Batches:   0%|          | 0/3446 [00:00<?, ?it/s]

In [6]:
# Instanciando modelo gerador de embeddings multilinguais
multilingual2_model = SentenceTransformer('all-MiniLM-L6-v2')

# Sugiro uso de GPUs se possível
X_train_multilingual2_embeddings = multilingual2_model.encode(
    X_train.texto.to_list(),
    show_progress_bar=True,
    normalize_embeddings=True
)

X_val_multilingual2_embeddings = multilingual2_model.encode(
    X_val.texto.to_list(),
    show_progress_bar=True,
    normalize_embeddings=True
)

# np.save("X_train_multilingual2_embeddings.npy", X_train_multilingual2_embeddings)
# np.save("X_val_multilingual2_embeddings.npy", X_val_multilingual2_embeddings)

# Se já possuir as embeddings em formato de arquivo .py comente as linhas acima e
# descomente a seguir
# X_train_multilingual2_embeddings = np.load("X_train_multilingual2_embeddings.npy")
# X_val_multilingual2_embeddings = np.load("X_val_multilingual2_embeddings.npy")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Treinamento e avaliação dos modelos

In [14]:
# Baixando dados de teste
!gdown 1QfrgY6HDhRk2XN_YESE36xzI7lqVawj7
!gdown 1-1t9vToIC9JnzS03mzwLLUS0xnS3egWG

Downloading...
From (original): https://drive.google.com/uc?id=1QfrgY6HDhRk2XN_YESE36xzI7lqVawj7
From (redirected): https://drive.google.com/uc?id=1QfrgY6HDhRk2XN_YESE36xzI7lqVawj7&confirm=t&uuid=5d7be04c-7e96-4d65-ac34-5d5054c8a6ab
To: /content/X_test.pkl
100% 253M/253M [00:03<00:00, 77.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-1t9vToIC9JnzS03mzwLLUS0xnS3egWG
To: /content/Y_test.pkl
100% 34.4M/34.4M [00:00<00:00, 97.6MB/s]


In [15]:
X_test = pd.read_pickle("X_test.pkl")
y_test = pd.read_pickle("Y_test.pkl")

X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [16]:
X_test.head()

,texto
0,DECISÃO: Trata-se de mandado de segurança impe...
1,"DESPACHO: Vistos, etc. A apreciação do pedido ..."
2,(PG/STF 112525/03) DECISÃO: O Senhor President...
3,DECISÃO UNIVERSIDADE – SISTEMA DE COTAS – RECU...
4,DECISÃO AÇÃO CAUTELAR – SUSPENSÃO. 1. Deferi a...


In [18]:
# Transformando vetores de classes para formato apropriado a esparso
s_Ytrain = sparse.csr_matrix(np.array(y_train))
s_Yval = sparse.csr_matrix(np.array(y_val))

### Treinando knn para embeddings multilinguais 1

In [ ]:
s_Xtrain = sparse.csr_matrix(X_train_multilingual_embeddings)
s_Xval = sparse.csr_matrix(X_val_multilingual_embeddings)

# Faixa de valores de k para testar
k_values = [10, 50, 100]

best_k = None
best_f1 = 0

for k in k_values:
    # Instancia o modelo com o valor atual de k
    classifier = MLkNN(k=k)

    # Treina o modelo nos dados de treinamento
    classifier.fit(s_Xtrain, s_Ytrain)

    # Faz predições no conjunto de validação
    predictions = classifier.predict(s_Xval)

    # Calcula o F1-Score no conjunto de validação
    f1 = f1_score(s_Yval, predictions, average="weighted")

    print(f"F1-Score para k={k}: {f1}")

    # Atualiza o melhor k com base no F1-Score
    if f1 > best_f1:
        best_f1 = f1
        best_k = k

print(f"Melhor k: {best_k} com F1-Score: {best_f1}")

# Treina o modelo final com o melhor k encontrado
final_classifier = MLkNN(k=best_k)
final_classifier.fit(s_Xtrain, s_Ytrain)

In [19]:
# Criando embbedings dos dados de treino
X_test_multilingual_embeddings = multilingual_model.encode(
    X_test.texto.to_list(),
    show_progress_bar=True,
    normalize_embeddings=True
)

np.save("X_test_multilingual_embeddings.npy", X_test_multilingual_embeddings)

# Se já possuir os embeddings descomentar abaixo:
# X_test_multilingual_embeddings = np.load("X_test_multilingual_embeddings.npy")

# s_Xtest = sparse.csr_matrix(X_test_multilingual_embeddings)

# # Avalia o modelo final no conjunto de teste
# test_predictions = final_classifier.predict(s_Xtest)

# print("F1-Score no Teste (weighted):", f1_score(s_Ytest, test_predictions, average="weighted"))
# print("Relatório de Classificação no Teste:\n", classification_report(s_Ytest, test_predictions))

Batches:   0%|          | 0/1725 [00:00<?, ?it/s]

In [ ]:
del s_Xtrain
del s_Xval
del s_Ytrain
del s_Yval
del classifier
del predictions
gc.collect()

### Avaliando knn para embeddings multilinguais 2

In [ ]:
s_Xtrain = sparse.csr_matrix(X_train_multilingual2_embeddings)
s_Xval = sparse.csr_matrix(X_val_multilingual2_embeddings)

# Faixa de valores de k para testar
k_values = [10, 50, 100]

best_k = None
best_f1 = 0

for k in k_values:
    # Instancia o modelo com o valor atual de k
    classifier = MLkNN(k=k)

    # Treina o modelo nos dados de treinamento
    classifier.fit(s_Xtrain, s_Ytrain)

    # Faz predições no conjunto de validação
    predictions = classifier.predict(s_Xval)

    # Calcula o F1-Score no conjunto de validação
    f1 = f1_score(s_Yval, predictions, average="weighted")

    print(f"F1-Score para k={k}: {f1}")

    # Atualiza o melhor k com base no F1-Score
    if f1 > best_f1:
        best_f1 = f1
        best_k = k

print(f"Melhor k: {best_k} com F1-Score: {best_f1}")

# Treina o modelo final com o melhor k encontrado
final_classifier = MLkNN(k=best_k)
final_classifier.fit(s_Xtrain, s_Ytrain)

In [20]:
# Criando embbedings dos dados de treino
X_test_multilingual2_embeddings = multilingual2_model.encode(
    X_test.texto.to_list(),
    show_progress_bar=True,
    normalize_embeddings=True
)

np.save("X_test_multilingual2_embeddings.npy", X_test_multilingual2_embeddings)

# Se já possuir os embeddings descomentar abaixo:
# X_test_multilingual2_embeddings = np.load("X_test_multilingual2_embeddings.npy")

s_Xtest = sparse.csr_matrix(X_test_multilingual2_embeddings)

# Avalia o modelo final no conjunto de teste
test_predictions = final_classifier.predict(s_Xtest)

print("F1-Score no Teste (weighted):", f1_score(s_Ytest, test_predictions, average="weighted"))
print("Relatório de Classificação no Teste:\n", classification_report(s_Ytest, test_predictions))

Batches:   0%|          | 0/1725 [00:00<?, ?it/s]

In [ ]:
del s_Xtrain
del s_Xval
del s_Ytrain
del s_Yval
del classifier
del predictions
gc.collect()

### Avaliando knn matriz tf-idf

In [ ]:
y_train_sparse = csr_matrix(y_train)
y_val_sparse = csr_matrix(y_val)

# Instanciando o modelo
mlknn = MLkNN(k=13)

# Treinando o modelo
mlknn.fit(X_train_tf, y_train_sparse)

# Fazendo previsões
predictions = mlknn.predict(X_val_tf)

# Avaliando o modelo
print("F1-Score (weighted):", f1_score(y_val_sparse, predictions, average="weighted"))
print("Classification Report:\n", classification_report(y_val_sparse, predictions))